<a href="https://colab.research.google.com/github/inyong37/Study/blob/master/_Framework/Keras/Keras_Developer_Guide_13_Transfer_Learning_%26_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Title: Keras Developer Guide - Transfer Learning & Fine-tuning
# Re-Author: Inyong Hwang
# Date: 2021-12-30-Thu.
# Reference: https://keras.io/guides/

import numpy as np
import tensorflow as tf
from tensorflow import keras

'''
# Introduction
The most common incarnation of transfer learning in the context of deep learning is the following workflow:
1. Take layers from a previously trained model.
2. Freeze them, so as to avoid destroying any of the information they contain during future training rounds.
3. Add some new, trainable layers on top of the frozen layers. They will learn to turn the old features into predictions on a new dataset.
4. Train the new layers on your dataset.

A last, optional step, is fine-tuning, which consists of unfreezing the entire model you obtained above (or part of it), and re-training it on the new data with a very low learning rate.
This can potentially achieve meaningful improvements, by incrementally adapting the pretrained features to the new data.
'''

# Freezing layers: understanding the trainable attribute
layer = keras.layers.Dense(3)
layer.build((None, 4)) # Create the weights

print('weights:', len(layer.weights))
print('trainable_weights:', len(layer.trainable_weights))
print('non_trainable_weights:', len(layer.non_trainable_weights))